# Regresión penalizada con Lasso

Dataset: California housing

Target: Mediana de los valores de las casas 

Predictoras: variables numéricas y categóricas

Se determina a los hiperparámetros por medio de validación cruzada

# Librerias

In [1]:
import pandas as pd
import numpy as np
from pandas.core.common import flatten
from plotnine import *
from array import *
import scipy.stats as stats
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

# Los datos

Carga de datos

In [2]:
path = r'~/Documents/GitHub/Supervisado/dataset/housing.csv'
housing = pd.read_csv(path)

Variable respuesta y descriptoras

In [3]:
y = housing["median_house_value"].copy()
housing = housing.drop("median_house_value", axis=1)

Variables numéricas

In [4]:
housing_num = housing.drop("ocean_proximity", axis=1)

El imputador

In [5]:
from sklearn.impute import SimpleImputer

El estandarizador

In [6]:
from sklearn.preprocessing import StandardScaler

La clase pipeline para variables numéricas

In [7]:
from sklearn.pipeline import Pipeline

Definición del pipeline que trabaja variables numéricas

In [8]:
num_pipeline = Pipeline([
        ("imputador", SimpleImputer(strategy="median")),
        ("std_scaler", StandardScaler()),
    ])

Clases para procesar a las variables categóricas

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

Definición del pipeline que trabaja variables numéricas y categóricas

Variables numéricas y categóricas

In [10]:
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

In [11]:
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(drop = "first"), cat_attribs),
    ])

Aplicación del fullpipeline

In [12]:
X = full_pipeline.fit_transform(housing)

# Busqueda de mejores hiper-parámetros

Establecer semilla para obtener el mismo resultado

In [13]:
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
from numpy import arange
from sklearn.model_selection import GridSearchCV

Creación del modelo

In [14]:
model = Lasso()

Métrica para evaluar al modelo

In [15]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)

Creación de la malla

In [16]:
grid = dict()
grid['alpha'] = arange(0.5, 1.5, 0.1)

Definición de la búsqueda

In [17]:
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

Búsqueda de mejores hiper-parámetros

In [18]:
results = search.fit(X, y)

Resumen de resultados

In [19]:
print('MAE: %.3f' % results.best_score_)

MAE: -49907.660


In [20]:
print('Config: %s' % results.best_params_)

Config: {'alpha': 0.5}


## Predicciones con los mejores hiper-parámetros

Se crea un modelo con los mejores hiper-parámetros

In [21]:
best_model = Lasso(alpha=0.5)

Para medir el comportamiento del modelo

In [22]:
cv1 = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)

Evaluación del mejor modelo

In [23]:
scores = cross_val_score(best_model, X, y, scoring='neg_mean_absolute_error', cv=cv1, n_jobs=-1)

Resultados de score positivos

In [24]:
scores = absolute(scores)

In [25]:
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean MAE: 49907.660 (1250.962)


Ajuste del mejor modelo

In [26]:
best_model.fit(X, y)

Lasso(alpha=0.5)

Un dato para la predicción

In [27]:
row = X[:1][0]
row

array([-1.32783522,  1.05254828,  0.98214266, -0.8048191 , -0.97247648,
       -0.9744286 , -0.97703285,  2.34476576,  0.        ,  0.        ,
        1.        ,  0.        ])

La predicción

In [28]:
y_pred = best_model.predict([row])
y_pred

array([408493.59252162])

In [29]:
y[:1]

0    452600.0
Name: median_house_value, dtype: float64

Elaborado por Jairo Rojas